In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import torch
import torchvision
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
from torch.autograd import Variable


In [ ]:
from model import Decoder
from data_utils import features_dataset, orientations_iterator
from train_utils import train

## Get orientation dataloader

##### We'll get the orientation maps as we load the ImageNet images

In [ ]:
KERNEL_SIZE = 30
assert KERNEL_SIZE in (5,10,15,20,25,30)

### Get X data loader

In [ ]:

LAYER_NUMBER = 24
assert LAYER_NUMBER in (24,31)

feature_num_dict = {24: 100352, 31 :512*7*7}
num_features = feature_num_dict[LAYER_NUMBER]

In [ ]:


# features_data_train = features_dataset('fast_data/features/vgg_maxpool{}.h5'.format(LAYER_NUMBER), train = True)
features_data_valid = features_dataset('fast_data/features/vgg_maxpool{}.h5'.format(LAYER_NUMBER), train = False)

In [ ]:
# num_ftrs = 512*7*7#100352
# model = color_predictor(num_ftrs).cuda()

model.load_state_dict(torch.load('/home/abenjamin/DNN_illusions/data/models/aritrained_kernel5_0.pt'))

# print('done loading')

Look at some predictions

In [ ]:
valdir = '/data2/imagenet/val'
# Load the images for visualizing
val_dataset_nonnormalized = datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]))

In [ ]:
class SubsetSampler(torch.utils.data.Sampler):
    r"""Samples elements randomly from a given list of indices, without replacement.

    Arguments:
        indices (sequence): a sequence of indices
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)

In [ ]:
from colorspacious import cspace_convert


from matplotlib.colors import ListedColormap

def plot_viewable(x, y):
    
    magnitude = torch.sqrt(x**2+y**2)
    magnitude /= torch.max(magnitude)
    
    angle = torch.atan2(y,x)
    
    color_circle = np.ones((256,3))*66
    color_circle[:,1] = np.ones((256))*44

    color_circle[:,2] = np.arange(0,360,360/256)
    
    color_circle_rgb = cspace_convert(color_circle, "JCh","sRGB1")
    cm = ListedColormap(color_circle_rgb)

    a =plt.imshow(angle, 
                  cmap= cm,
                  vmin = -np.pi,
                  vmax = np.pi)

    plt.imshow(1-magnitude,vmin=0,vmax=1, alpha=.6)
    
    cbar = plt.colorbar(a, aspect = 10, fraction = .07)
    cbar.ax.set_ylabel('Phase [pi]')
#     plt.show()

In [ ]:
# get predicted outputs for some inputs

TEST_BATCH_SIZE = 64
BATCH_SIZE = 2
LOG_INTERVAL = 100
EPOCHS = 10


features_loader_2 = torch.utils.data.DataLoader(features_data_valid,
    batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True, drop_last = True)


orientation_loader_10 = orientations_iterator('data/features/orientations/{}/all_images.h5'.format(10), 
                                                      BATCH_SIZE, train = False)
orientation_loader_30 = orientations_iterator('data/features/orientations/{}/all_images.h5'.format(30), 
                                                      BATCH_SIZE, train = False)


image_loader = torch.utils.data.DataLoader(val_dataset_nonnormalized,
        batch_size=BATCH_SIZE, shuffle=False,sampler = SubsetSampler(range(48000,50000)),
        num_workers=1, pin_memory=True)

for batch_idx, ((feats, _), orients10,orients30, (images, _)) in enumerate(zip(features_loader_2,
                                                                        orientation_loader_10,orientation_loader_30,
                                                                       image_loader)):
        data1 = feats.cuda()
        target = orients30.cuda()
        data1 =  Variable(data1)
        
        
        
#         output = model(data1)
        

        for index in range(BATCH_SIZE):
            plt.figure(figsize=(10,10))
#             plt.subplot(221)
#             x,y = output[index].detach().cpu()
#             plot_viewable(x, y)
#             plt.title("Model output")
#             plt.axis("off")
            
            plt.subplot(222)
            x,y = target[index].detach().cpu()
            plot_viewable(x, y)
            plt.title("Loaded orientations: 30")
            plt.axis("off")
                        
            plt.subplot(224)
            x,y = orients10[index].detach().cpu()
            plot_viewable(x, y)
            plt.title("Loaded orientations: 10")
            plt.axis("off")
            
            plt.subplot(223)
            image = images[index]
            image = np.squeeze(np.moveaxis(image.numpy(),0,-1))
            plt.imshow(image)
            plt.title("Da pic")
            plt.axis("off")
    
        
         
#             magnitude, angle = to_viewable_image(three_d_output)
#             magnitude1, angle1 = to_viewable_image(three_d_orient)
#             print("Output Image")
#             plot_viewable(magnitude, angle)
#             print("Original Orientation")
#             plot_viewable(magnitude1, angle1)
            
        break
print("done")
        



# compare with the function above




